<a href="https://colab.research.google.com/github/gemjk07/ecommerce_project/blob/main/Create_bigquery_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# pip install google-cloud-bigquery
# pip install db-dtypes
# pip install pandas_gbq

In [42]:
import os
import glob
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Path to the service account key JSON file
key_path = "/content/metacode-13-5a63b6c2a5fd.json"

# Create a Credentials object
credentials = service_account.Credentials.from_service_account_file(key_path)


# Data Upload on Big *Query*

In [47]:
df_view = pd.read_csv('/content/drive/MyDrive/data/data2/beauty_view_product_201910_202002.csv')
df_add_cart = pd.read_csv('/content/drive/MyDrive/data/data2/beauty_add_to_cart__201910_202002.csv')
df_remove_cart = pd.read_csv('/content/drive/MyDrive/data/data2/beauty_remove_from_cart__201910_202002.csv')
df_complete_purchase = pd.read_csv('/content/drive/MyDrive/data/data2/beauty_complete_purchase_201910_202002.csv')

In [48]:
# Check the number of rows and columns in the data
print(df_view.shape, df_add_cart.shape, df_remove_cart.shape, df_complete_purchase.shape)

(1931562, 9) (1153663, 9) (795932, 9) (257399, 9)


In [49]:
# All the data are showing
df_view.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-02 16:51:46+00:00,view,5826988,1487580005671109489,NaN,masura,1.73,486531759,a695773d-deeb-45cb-b9b1-6ee91c99e96f
1,2019-10-29 15:22:14+00:00,view,5885428,1487580007281722301,NaN,grattol,12.06,565450708,d2b747d5-cdc2-49ac-9e37-770b78306c20
2,2019-10-20 10:37:40+00:00,view,5887293,1487580013128581300,NaN,NaN,3.14,562206611,2a66e7fa-edc1-40d2-9eaf-50f364e587b8
3,2019-10-03 20:01:59+00:00,view,5861445,1487580013841613016,NaN,farmavita,5.56,524175484,4f38b717-5af8-4c1d-80c5-079415f979c8
4,2019-10-10 17:40:55+00:00,view,5635068,1487580005754995573,NaN,NaN,4.44,482929978,4dbb4d89-9d56-3b85-d8b0-43f572b96f9a


In [50]:
df_add_cart.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-19 10:23:07+00:00,cart,5784039,1487580006551913373,NaN,NaN,3.02,561861301,d5e43488-fb23-4ba2-9ba4-e2466578a55d
1,2019-10-02 09:15:19+00:00,cart,5881894,1487580012096782476,NaN,NaN,3.89,555947030,01581986-f0bb-4be6-8553-d90cbe9acc10
2,2019-10-07 21:34:45+00:00,cart,5700046,1487580009286598681,NaN,runail,0.40,374211459,2857db91-efec-4771-8e89-66d5337261b7
3,2019-10-26 23:07:46+00:00,cart,5823502,1487580004916134735,NaN,NaN,4.70,539691934,2c63086a-4bf9-4d08-2ede-b81d6a49f1bb
4,2019-10-28 10:09:58+00:00,cart,5812216,1487580005427839846,NaN,irisk,2.48,564930298,dd1e0e29-a373-4a4f-a014-a9ccc64cf895


In [51]:
df_remove_cart.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-10 08:15:59+00:00,remove_from_cart,5652771,1487580013950664926,NaN,NaN,13.49,557510291,d00bcd26-e37e-4318-ab7a-f3b55a74eebe
1,2019-10-04 17:43:06+00:00,remove_from_cart,5796994,1487580005671109489,NaN,masura,1.73,526073142,6c2a347f-c50c-446e-9817-7703873b979a
2,2019-10-22 12:00:20+00:00,remove_from_cart,5835856,1715102762414375164,NaN,NaN,4.44,537046919,f2c97fe9-6cb2-40f5-b936-43f736b222bd
3,2019-10-23 10:32:24+00:00,remove_from_cart,5815664,1487580006317032337,NaN,NaN,0.92,505419579,82114d16-4fcc-4513-aa20-2db7d656d584
4,2019-10-28 17:26:44+00:00,remove_from_cart,5885421,1487580005092295511,NaN,grattol,6.27,480237908,fe7dfdbf-439f-447a-a4e7-bdc7d6645a78


In [52]:
df_complete_purchase.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-28 09:41:40+00:00,purchase,5753837,1487580007936033754,NaN,zinger,5.52,564971889,dd7b448d-6561-4db7-a63f-b4221f1270ad
1,2019-10-19 18:29:02+00:00,purchase,5715532,1487580006300255120,NaN,NaN,95.71,561996225,fcf69a0f-0b92-4295-bc13-8e08c20cdf46
2,2019-10-30 10:58:47+00:00,purchase,5836844,1487580008187692007,NaN,pnb,5.79,565720702,6f1348f0-6dc3-433e-8b20-6191e28b64a0
3,2019-10-26 18:29:28+00:00,purchase,5649418,1487580009387261981,NaN,NaN,4.76,536033806,77e35af4-da52-447b-aa04-387d7a8e221e
4,2019-10-04 14:33:38+00:00,purchase,5588,1487580006132482952,NaN,runail,4.84,531394068,d6bc9d0a-6d66-4e7f-a520-24d475f07240


In [53]:
df_view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1931562 entries, 0 to 1931561
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 132.6+ MB


In [55]:
#Change data type of event_time to datetime from object
df_view['event_time'] = pd.to_datetime(df_view['event_time'])
df_add_cart['event_time'] = pd.to_datetime(df_add_cart['event_time'])
df_remove_cart['event_time'] = pd.to_datetime(df_remove_cart['event_time'])
df_complete_purchase['event_time'] = pd.to_datetime(df_complete_purchase['event_time'])

#Check the data
print("data type of event_time is:", df_view['event_time'].dtype)
print("data type of event_time is:", df_add_cart['event_time'].dtype)
print("data type of event_time is:", df_remove_cart['event_time'].dtype)
print("data type of event_time is:", df_complete_purchase['event_time'].dtype)

data type of event_time is: datetime64[ns, UTC]
data type of event_time is: datetime64[ns, UTC]
data type of event_time is: datetime64[ns, UTC]
data type of event_time is: datetime64[ns, UTC]


In [62]:
# BigQuery project path
project_id = 'metacode-13'

# Create a BigQuery client
client = bigquery.Client(project='metacode-13', credentials=credentials)

# Define a function to create a table
def create_table(df, table_name):
    dataset_table = f'ecommerce2.{table_name}'
    table_id = f"{project_id}.{dataset_table}"

    # Automatically generate schema
    schema = [
        bigquery.SchemaField("event_time", "TIMESTAMP"),
        bigquery.SchemaField("event_type", "STRING"),
        bigquery.SchemaField("product_id", "INTEGER"),
        bigquery.SchemaField("category_id", "INTEGER"),
        bigquery.SchemaField("category_code", "STRING"),
        bigquery.SchemaField("brand", "STRING"),
        bigquery.SchemaField("price", "FLOAT"),
        bigquery.SchemaField("user_id", "INTEGER"),
        bigquery.SchemaField("user_session", "STRING"),  # Date field for partitioning
    ]

    # Table configuration
    table = bigquery.Table(table_id, schema=schema)

    # Partition settings (date partition)
    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="event_time",
    )

    # Create table
    table = client.create_table(table)
    print(f"Table created: {table.table_id}")

    # Upload data
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df, table, job_config=job_config)
    job.result()
    print(f"Data upload complete: {table.table_id}")

In [63]:
# Create tables and upload data for each dataframe
create_table(df_view, "view_product")
create_table(df_add_cart, "add_to_cart")
create_table(df_remove_cart, "remove_from_cart")
create_table(df_complete_purchase, "complete_purchase")

Table created: view_product
Data upload complete: view_product
Table created: add_to_cart
Data upload complete: add_to_cart
Table created: remove_from_cart
Data upload complete: remove_from_cart
Table created: complete_purchase
Data upload complete: complete_purchase
